# A1 : Search Engine (Negative Sampling)

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib
import matplotlib.pyplot as plt

In [2]:
np.__version__, torch.__version__, matplotlib.__version__

('1.25.2', '2.1.0', '3.7.2')

## 1. Data Loader

In [3]:
# read the nltk dataset = rural
txt_file = './abc/rural.txt'

with open(txt_file, 'r', encoding = 'utf-8') as file:
        text = file.read()
        
# Split the dataset into paragraphs based on double line breaks : to get one paragraph in a list item ['paragraph1', 'paragraph']
paragraphs = [paragraph.strip() for paragraph in text.split('\n\n')]

In [4]:
for i in range (len(paragraphs)):
    # Replace newline characters with spaces
    paragraphs[i] = paragraphs[i].replace('\n', ' ')

In [5]:
paragraphs

['PM denies knowledge of AWB kickbacks The Prime Minister has denied he knew AWB was paying kickbacks to Iraq despite writing to the wheat exporter asking to be kept fully informed on Iraq wheat sales. Letters from John Howard and Deputy Prime Minister Mark Vaile to AWB have been released by the Cole inquiry into the oil for food program. In one of the letters Mr Howard asks AWB managing director Andrew Lindberg to remain in close contact with the Government on Iraq wheat sales. The Opposition\'s Gavan O\'Connor says the letter was sent in 2002, the same time AWB was paying kickbacks to Iraq though a Jordanian trucking company. He says the Government can longer wipe its hands of the illicit payments, which totalled $290 million. "The responsibility for this must lay may squarely at the feet of Coalition ministers in trade, agriculture and the Prime Minister," he said. But the Prime Minister says letters show he was inquiring about the future of wheat sales in Iraq and do not prove the 

### 1.1 Tokenization

In [6]:
# 1. tokenization
corpus = [sent.split(" ") for sent in paragraphs]
corpus

[['PM',
  'denies',
  'knowledge',
  'of',
  'AWB',
  'kickbacks',
  'The',
  'Prime',
  'Minister',
  'has',
  'denied',
  'he',
  'knew',
  'AWB',
  'was',
  'paying',
  'kickbacks',
  'to',
  'Iraq',
  'despite',
  'writing',
  'to',
  'the',
  'wheat',
  'exporter',
  'asking',
  'to',
  'be',
  'kept',
  'fully',
  'informed',
  'on',
  'Iraq',
  'wheat',
  'sales.',
  'Letters',
  'from',
  'John',
  'Howard',
  'and',
  'Deputy',
  'Prime',
  'Minister',
  'Mark',
  'Vaile',
  'to',
  'AWB',
  'have',
  'been',
  'released',
  'by',
  'the',
  'Cole',
  'inquiry',
  'into',
  'the',
  'oil',
  'for',
  'food',
  'program.',
  'In',
  'one',
  'of',
  'the',
  'letters',
  'Mr',
  'Howard',
  'asks',
  'AWB',
  'managing',
  'director',
  'Andrew',
  'Lindberg',
  'to',
  'remain',
  'in',
  'close',
  'contact',
  'with',
  'the',
  'Government',
  'on',
  'Iraq',
  'wheat',
  'sales.',
  'The',
  "Opposition's",
  'Gavan',
  "O'Connor",
  'says',
  'the',
  'letter',
  'was',
 

### 1.2 numericalization

In [7]:
# get unique word

# list comprehension for getting words
flatten = lambda l: [item for sublist in l for item in sublist]

# getting unique word and store as a list
vocab = list(set(flatten(corpus)))
vocab

['',
 '32nd',
 'cereal',
 'products,"',
 "'botched'",
 'vaccine,',
 'Argentina,',
 '"Plastics',
 'Bonlac',
 'figure,"',
 'Angus',
 'Resources,',
 'organising',
 'hungry.',
 'tackled',
 'nurses.',
 'Naomi',
 'increasing.',
 'most',
 'approached',
 '$27.36',
 'transportation',
 'vets',
 "fuller's",
 'lighten',
 'consequences.',
 'four-metre',
 'Gubler',
 "ministers'",
 'Processing',
 'Tourism',
 'fry',
 'Jury',
 'Eden',
 'marked',
 'Stallholders',
 'inflows.',
 'bushland',
 'Force',
 'Darley,',
 'hygienic',
 'talked',
 'consumables',
 'Murgon',
 'Gen',
 'gold.',
 'AUSVEG,',
 'Numbers',
 'autumn',
 "'caution'",
 'Zircon',
 'nor',
 'beekeeper',
 'inaccurate',
 'depots',
 '$316,000',
 '6.2',
 'Hynd',
 'townsfolk,',
 'Handford',
 'reserve',
 'Roach',
 'nests',
 'duress,"',
 'long-delayed',
 'hole',
 'Patchy',
 'holes',
 'Brennan',
 'Young',
 'Upper',
 'herpes-like',
 'extension',
 'Curd',
 'stand-by',
 'sometime',
 'Costello,',
 'McKew',
 'ignoring',
 '782',
 'instead',
 'extraction',
 '$9.0

In [8]:
# add <UNK> to a dictionary vocab
vocab.append('<UNK>')

In [9]:
# numericalization: assign index to each word
word2index = {w:idx for idx, w in enumerate(vocab)}
word2index

{'': 0,
 '32nd': 1,
 'cereal': 2,
 'products,"': 3,
 "'botched'": 4,
 'vaccine,': 5,
 'Argentina,': 6,
 '"Plastics': 7,
 'Bonlac': 8,
 'figure,"': 9,
 'Angus': 10,
 'Resources,': 11,
 'organising': 12,
 'hungry.': 13,
 'tackled': 14,
 'nurses.': 15,
 'Naomi': 16,
 'increasing.': 17,
 'most': 18,
 'approached': 19,
 '$27.36': 20,
 'transportation': 21,
 'vets': 22,
 "fuller's": 23,
 'lighten': 24,
 'consequences.': 25,
 'four-metre': 26,
 'Gubler': 27,
 "ministers'": 28,
 'Processing': 29,
 'Tourism': 30,
 'fry': 31,
 'Jury': 32,
 'Eden': 33,
 'marked': 34,
 'Stallholders': 35,
 'inflows.': 36,
 'bushland': 37,
 'Force': 38,
 'Darley,': 39,
 'hygienic': 40,
 'talked': 41,
 'consumables': 42,
 'Murgon': 43,
 'Gen': 44,
 'gold.': 45,
 'AUSVEG,': 46,
 'Numbers': 47,
 'autumn': 48,
 "'caution'": 49,
 'Zircon': 50,
 'nor': 51,
 'beekeeper': 52,
 'inaccurate': 53,
 'depots': 54,
 '$316,000': 55,
 '6.2': 56,
 'Hynd': 57,
 'townsfolk,': 58,
 'Handford': 59,
 'reserve': 60,
 'Roach': 61,
 'nests

In [10]:
# index2word
index2word = {k:v for v,k in word2index.items()}
index2word

{0: '',
 1: '32nd',
 2: 'cereal',
 3: 'products,"',
 4: "'botched'",
 5: 'vaccine,',
 6: 'Argentina,',
 7: '"Plastics',
 8: 'Bonlac',
 9: 'figure,"',
 10: 'Angus',
 11: 'Resources,',
 12: 'organising',
 13: 'hungry.',
 14: 'tackled',
 15: 'nurses.',
 16: 'Naomi',
 17: 'increasing.',
 18: 'most',
 19: 'approached',
 20: '$27.36',
 21: 'transportation',
 22: 'vets',
 23: "fuller's",
 24: 'lighten',
 25: 'consequences.',
 26: 'four-metre',
 27: 'Gubler',
 28: "ministers'",
 29: 'Processing',
 30: 'Tourism',
 31: 'fry',
 32: 'Jury',
 33: 'Eden',
 34: 'marked',
 35: 'Stallholders',
 36: 'inflows.',
 37: 'bushland',
 38: 'Force',
 39: 'Darley,',
 40: 'hygienic',
 41: 'talked',
 42: 'consumables',
 43: 'Murgon',
 44: 'Gen',
 45: 'gold.',
 46: 'AUSVEG,',
 47: 'Numbers',
 48: 'autumn',
 49: "'caution'",
 50: 'Zircon',
 51: 'nor',
 52: 'beekeeper',
 53: 'inaccurate',
 54: 'depots',
 55: '$316,000',
 56: '6.2',
 57: 'Hynd',
 58: 'townsfolk,',
 59: 'Handford',
 60: 'reserve',
 61: 'Roach',
 62: 'n

## 2. Preparation for the training data

In [11]:
def random_batch(batch_size, corpus):

    # define a list for storing [center,outside] pair
    skipgrams = []

    # loop each word sequence
    for sent in corpus:
        
        for i in range(2, len(sent)-2):
            
            # assign center word
            center_word = word2index[sent[i]]
            
            # assign outside word=4 (ws = 2)
            outside_word = [word2index[sent[i-2]], word2index[sent[i-1]], word2index[sent[i+1]], word2index[sent[i+2]]]
            
            # for each of these two outside words, we gonna pair (center,outside) and append to a list
            for each_outside in outside_word:
                skipgrams.append([center_word, each_outside])
                
    # randomly select 2 pair among the data
    random_index = np.random.choice(range(len(skipgrams)), batch_size, replace = False)
    
    random_inputs = []
    random_labels = []
    
    for i in random_index:
        random_inputs.append([skipgrams[i][0]]) # center_word
        random_labels.append([skipgrams[i][1]]) # outside_word
        
    return np.array(random_inputs), np.array(random_labels)

## 3. Negative Sampling

### Unigram Distribution

$$P(w) = U(w)^{3/4} / Z

In [12]:
z = 0.001 # according to the papaer

In [13]:
# count
from collections import Counter

word_count = Counter(flatten(corpus))
word_count # {'apple': 10, 'orange' : 5}

# get the total number of words
num_total_words = sum([c for w,c in word_count.items()])
print(num_total_words)

301279


In [14]:
# U(w)

unigram_table = []

for v in vocab:
    uw = word_count[v] / num_total_words
    uw_alpha = int((uw ** 0.75) / z)
    # print(v, uw)
    # print(v, uw_alpha)
    # print([v] * uw_alpha)
    # print("---")
    unigram_table.extend([v] * uw_alpha) 
    
Counter(unigram_table)

Counter({'the': 107,
         'to': 75,
         'of': 62,
         'and': 53,
         'in': 50,
         'a': 48,
         'is': 38,
         'for': 33,
         'says': 31,
         'are': 26,
         'has': 26,
         'that': 25,
         'have': 24,
         'The': 24,
         'from': 23,
         'on': 23,
         'be': 23,
         'he': 23,
         'said.': 22,
         'will': 22,
         'it': 20,
         'at': 18,
         'with': 18,
         'been': 16,
         'not': 15,
         'by': 14,
         'this': 13,
         'we': 13,
         'as': 13,
         'they': 13,
         'Australian': 12,
         'more': 11,
         'but': 11,
         'up': 11,
         'an': 11,
         'Australia': 11,
         'their': 11,
         'farmers': 10,
         'water': 10,
         'was': 10,
         'which': 10,
         'South': 10,
         'about': 10,
         'there': 10,
         'growers': 9,
         'per': 9,
         'Government': 9,
         'its': 9,
       

## 4. Model

In [15]:
def prepare_sequence (seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index['<UNK>'],seq))
    return torch.LongTensor(idxs)

In [28]:
import random

def negative_sampling(targets, unigram_table, k):
    
    batch_size  = targets.shape[0]
    neg_samples = []
    
    for i in range(batch_size): # (1,k) # (batch_size, k)
        target_index = targets[i].item()
        nsample = []
        while (len(nsample) < k):
            neg = random.choice(unigram_table)
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        neg_samples.append(prepare_sequence(nsample, word2index).reshape(1,-1))
            
        
    return torch.cat(neg_samples) # [batch_size,k]
    

#### testing

In [17]:
batch_size = 2
x,y = random_batch(batch_size, corpus)
x_tensor = torch.LongTensor(x)
y_tensor = torch.LongTensor(y)

In [18]:
k = 5
neg_samples = negative_sampling(y_tensor, unigram_table, k)

In [19]:
print(neg_samples.shape)
print(neg_samples[0])
print(neg_samples[1])
print(y)

torch.Size([2, 5])
tensor([11472, 14427,  1140,  4211, 25041])
tensor([ 8063, 24362,   257, 17022, 11912])
[[14746]
 [ 5855]]


#### Model

In [20]:
class SkipgramNeg (nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(SkipgramNeg, self).__init__()
        self.embedding_center  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside = nn.Embedding(voc_size, emb_size)
        self.logsigmoid        = nn.LogSigmoid()
    
    def forward(self, center, outside, negative_words):
        
        # center, outside : (bs, 1)
        # negative : (bs, k)
        
        center_embed  = self.embedding_center(center) # (bs, 1, emb_size)
        outside_embed = self.embedding_outside(outside) #(bs, 1, emb_size)
        neg_embed     = self.embedding_outside(negative_words) # (bs, k, emb_size)
        
        uovc          = outside_embed.bmm(center_embed.transpose(1,2)).squeeze(2) # (bs,1)
        ukvc          = -neg_embed.bmm(center_embed.transpose(1,2)).squeeze(2) #(bs,k)
        ukvc_sum      = torch.sum(ukvc, 1).reshape(-1,1) # sum across k , reshape>> (batch_size,1)
        
        loss = self.logsigmoid(uovc) + self.logsigmoid(ukvc_sum)
        
        return -torch.mean(loss)        
        

#### test the model

In [21]:
emb_size = 2
voc_size = len(vocab)
model = SkipgramNeg(voc_size, emb_size)

In [22]:
neg_samples 

tensor([[11472, 14427,  1140,  4211, 25041],
        [ 8063, 24362,   257, 17022, 11912]])

In [23]:
loss = model(x_tensor, y_tensor, neg_samples)

In [24]:
loss

tensor(0.8713, grad_fn=<NegBackward0>)

## 5.Train

In [30]:
import time

# for recording the training time for each epoch
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time # get the total taken timestamp
    elapsed_mins = int(elapsed_time / 60) # get the min
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60)) # get the sec
    return elapsed_mins, elapsed_secs

In [26]:
k = 5
emb_size = 2
voc_size = len(vocab)
model    = SkipgramNeg(voc_size, emb_size)
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

In [31]:
num_epochs = 5
# num_epochs = 50
loss_ar = []
start = time.time()

for epoch in range(num_epochs):
    
    # get batch
    input_batch, label_batch = random_batch(batch_size, corpus)
    input_tensor = torch.LongTensor(input_batch)
    label_tensor = torch.LongTensor(label_batch)
    
    # predict
    neg_samples = negative_sampling(label_tensor, unigram_table, k)
    loss = model (input_tensor, label_tensor, neg_samples)
    
    # backpropagate
    optimizer.zero_grad()
    loss.backward()
    
    # update alpha
    optimizer.step()
    
    # record loss
    loss_ar.append(loss)
    
    # print loss
    if (epoch+1) % 1000 == 0:
        print(f"Epoch {epoch+1:6.0f} | Loss {loss:2.6f}")

end = time.time()
    

Epoch      1 | Loss 1.003616
Epoch      2 | Loss 2.183491
Epoch      3 | Loss 1.416813
Epoch      4 | Loss 1.329990
Epoch      5 | Loss 1.825361


In [34]:
total_training_time = epoch_time(start,end)
print(f"total_training_time: {total_training_time[0]}m : {total_training_time[1]}s")

total_training_time: 0m : 3s


In [35]:
print(loss)

tensor(1.8254, grad_fn=<NegBackward0>)


## 6. Plotting the embeddings

## 7. Cosine Similarity